<a href="https://colab.research.google.com/github/anhnguyenvv/NLP-A/blob/main/project/ai_ali.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install gradio -q
!pip install openai -q
!pip install gtts -q
!pip install pydub -q
!pip install config -q
!pip install git+https://github.com/openai/whisper.git -q
!sudo apt update && sudo apt install ffmpeg
!pip install gdown -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 229 kB in 1s (168 kB/s)
Reading package lists... Done
Building dependency tree... Done
Reading state inf

In [3]:
import gradio as gr
import config
from gtts import gTTS
import os
import subprocess
from pydub import AudioSegment
import math
import whisper
from openai import OpenAI
import requests, gdown
import datetime

### load whisper model

In [4]:
import torch

# Checking if NVIDIA GPU is available
torch.cuda.is_available()
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
W_model = whisper.load_model("base", device=DEVICE)

In [16]:
#File Project1_Data.json
file_id = ["1jpM-ZP0h6OXDyuAfN1V6U7fvkdZuLLpU",
           "1vtdKzRvIwINdWEmnKG-U-VhFkrzTLDRL",
           "1v7SsfHr-slNIjVZ7tnliNJXYetVgwVya"
]
datafilename =[ "./Samples/SampleCall1.mp3",
        "./Samples/SampleCall2.mp3",
        "./Samples/SampleCall3.mp3"]
for i in range(len(file_id)):
   with open(datafilename[i], 'wb') as f:
     gdown.download(f"https://drive.google.com/uc?id={file_id[i]}", datafilename[i])

Downloading...
From: https://drive.google.com/uc?id=1jpM-ZP0h6OXDyuAfN1V6U7fvkdZuLLpU
To: /content/Samples/SampleCall1.mp3
100%|██████████| 7.69M/7.69M [00:00<00:00, 24.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1vtdKzRvIwINdWEmnKG-U-VhFkrzTLDRL
To: /content/Samples/SampleCall2.mp3
100%|██████████| 1.85M/1.85M [00:00<00:00, 182MB/s]
Downloading...
From: https://drive.google.com/uc?id=1v7SsfHr-slNIjVZ7tnliNJXYetVgwVya
To: /content/Samples/SampleCall3.mp3
100%|██████████| 2.86M/2.86M [00:00<00:00, 124MB/s]


In [18]:

# Set your OpenAI API key
client = OpenAI(api_key = "sk-oGx8ZhLZ0xMahXqAF3tGcsTQhfO7WGIiRC0UlwtNYKJU9pxu",
                base_url="https://api.chatanywhere.tech/v1")
mess = ""
with open('prompt.txt', 'r') as file:
    mess = file.read()
messages_system= {"role": "system",
                        "content": mess
                       }

# Transcribe audio using OpenAI's Whisper
def transcribe_audio(audio):
    segment_length = 600000
    # Open the audio file
    audio_file = AudioSegment.from_file(audio)
    # Get the duration of the audio file in milliseconds
    duration_ms = len(audio_file)
    # Calculate the number of segments needed
    num_segments = math.ceil(duration_ms / segment_length)
    # Create an empty string to hold the concatenated text
    all_text = ""
    # Split the audio file into segments
    for i in range(num_segments):
        start = i * segment_length
        end = min((i + 1) * segment_length, duration_ms)
        segment = audio_file[start:end]
        segment.export(f"segment_{i}.mp3", format="mp3")

    for i in range(num_segments):
        audio_file = open(f"segment_{i}.mp3", "rb")
        transcript = W_model.transcribe(f"segment_{i}.mp3")
        all_text += transcript['text']
    return all_text
import csv
def respond(
    message,
    history: list[tuple[str, str]],
    mic=None,
    text = "Simulator IELTS Speaking test",
):
    global messages_system
    result = ''
    messages = [messages_system]
    if len(history) != 0:
      messages.append({"role": "user", "content": text})
      with open('chat_transcript.csv', 'r') as f:
        history = csv.reader(f)

      for val in history:
          if val[0]:
              messages.append({"role": val[0], "content": val[1]})

      if mic is not None:
          audio = mic
      elif  len(message["files"])!= 0:
          audio = message["files"][0]
          #return message["files"][0]
      else:
          return "You must either provide a mic recording or a file"

      if audio:
        transcript = transcribe_audio(audio)
      else:
        transcript = message["text"]
      result = "Transcript: " + transcript+ "\n\n"

      messages.append({"role": "user", "content": transcript})
    else:
      messages.append({"role": "user", "content": text})

    response = ""
    response = client.chat.completions.create(
          messages= messages,
          model="gpt-3.5-turbo",
      )
    systems_message = response.choices[0].message.content
    messages.append({"role": "assistant", "content": systems_message})
    chat_transcript =list[tuple[str, str]]
    for message in messages:
        if message['role'] != 'system':
            chat_transcript.append([message['role'] , message['content']])
    with open('chat_transcript.csv', 'w') as f:
      writer = csv.writer(f)
      writer.writerows(chat_transcript)
    return systems_message +"\n\n"+ result

"""
"""
demo = gr.ChatInterface(
    respond,
    additional_inputs=[
        gr.Audio(sources="microphone", type="numpy", label="Speak here..."),
         gr.Radio(["Simulator IELTS Speaking test",
                                    "Analyze of an existing IELTS Speaking record" ,
                                   ], label=""),
    ],
    title="AI Auditor",
    description="AI Alliance for Audio Analytics Team. Our project's objective is to conduct a mock IELTS Speaking test, adhering closely to the IELTS standards.",
    textbox = gr.MultimodalTextbox(interactive=True, file_types=["audio"], placeholder="upload file..."),

    multimodal=True,
)


if __name__ == "__main__":
    demo.launch(debug = True)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://5e6fa623034fb6cce4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://5e6fa623034fb6cce4.gradio.live


Demo by gradio Interface

In [20]:
# Set your OpenAI API key
client = OpenAI(api_key = "sk-oGx8ZhLZ0xMahXqAF3tGcsTQhfO7WGIiRC0UlwtNYKJU9pxu",
                base_url="https://api.chatanywhere.tech/v1")
with open('prompt.txt', 'r') as file:
    mess = file.read()
messages_system= {"role": "system",
                        "content": mess
                       }
# Transcribe audio using OpenAI's Whisper
def transcribe_audio(audio):
    segment_length = 600000
    # Open the audio file
    audio_file = AudioSegment.from_file(audio)
    # Get the duration of the audio file in milliseconds
    duration_ms = len(audio_file)
    # Calculate the number of segments needed
    num_segments = math.ceil(duration_ms / segment_length)
    # Create an empty string to hold the concatenated text
    all_text = ''
    # Split the audio file into segments
    for i in range(num_segments):
        start = i * segment_length
        end = min((i + 1) * segment_length, duration_ms)
        segment = audio_file[start:end]
        segment.export(f"segment_{i}.mp3", format="mp3")


    for i in range(num_segments):
        audio_file = open(f"segment_{i}.mp3", "rb")
        transcript = W_model.transcribe(f"segment_{i}.mp3")
        all_text += transcript['text']

    return all_text

# Analyze transcript using OpenAI's GPT-3
def analyze_transcript(transcript):
    global messages_system
    messages=[]
    messages.append(messages_system)
    messages.append({"role": "user", "content": transcript})
    response = client.chat.completions.create(
          messages= messages,
          model="gpt-3.5-turbo",
      )

    systems_message = response.choices[0].message.content
    messages.append({"role": "assistant", "content": systems_message})

    chat_transcript = ""
    for message in messages:
        if message['role'] != 'system':
            chat_transcript += message['role'] + ": " + message['content'] + "\n\n"
    result = "Transcript: " + transcript['text'] + "\n\n"
    return systems_message +"\n\n"+ result
# Define the transcription and analysis function
def transcribe_and_analyze(mic=None, file=None):
    if mic is not None:
        audio = mic
    elif file is not None:
        audio = file
    else:
        return "You must either provide a mic recording or a file"
    # Transcribe the audio to text
    transcript = transcribe_audio(audio)
    # Analyze sentiment and summarize
    analysis = analyze_transcript(transcript)
    return analysis



In [ ]:

io = gr.Interface(fn=transcribe_and_analyze,
                      inputs=[
                          gr.Audio(sources="microphone", type="numpy", label="Speak here..."),
                          gr.Audio(sources="upload", type = "filepath"),
                      ],
                      outputs="text",
                      title="AI Auditor for",
                      description='''AI Alliance for Audio Analytics Team. Our project's objective is to
                      ''',
                      examples=[[None, "./Samples/SampleCall1.mp3"],
                                [None, "./Samples/SampleCall2.mp3"],
                                [None, "./Samples/SampleCall3.mp3"]],
                      )

io.launch(share=True, debug = True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://c1e636cf66b62a39bf.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 528, in process_events
    response = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 270, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1908, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1485, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/usr/local/lib/python3.10/dist-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/usr/local/lib/python3.10/dist-packa